Installed packages in virtual environment

In [ ]:
# %pip install pymongo
# %pip install pandas (will install numpy)
# %pip install google-cloud-bigquery


In [77]:
import pandas as pd
from pymongo import MongoClient
import numpy as np
import warnings

warnings.filterwarnings("ignore")

## 1. init connection to database
- Create 2 collections: business, reviews
- upload json files into their respective collections

In [3]:
client = MongoClient()
client = MongoClient('localhost', 27017)

In [4]:
yelp_db = client["Yelp_dataset"] 
business_collection = yelp_db['business'] 
review_collection = yelp_db['reviews']

## 2. Initialise tables

Currently, we need 4 tables
- Business table: { Business_id (PK) , Name , State, City , Stars, Review_count, weighted_review, Categories }
- Names table : {Name (PK) , Number of outlets, Average reviews , Total reviews} 
- Reviews table {Review_id (PK), Business_id (FK), Stars, Text}
- Categories table : {Category, Count (number of appearance), score (avg)}




TODO: calculate weighted reviews: https://medium.com/district-data-labs/computing-a-bayesian-estimate-of-star-rating-means-651496a890ab
https://www.codementor.io/@arpitbhayani/solving-an-age-old-problem-using-bayesian-average-15fy4ww08p#cumulative-rating 

#### a. Filter out all the restaurant establishments

In [78]:
business_records = []

for row in business_collection.find({'$or' : [{'categories' : {'$regex' : '.*Restaurants.*'}},{'categories' : {'$regex' : '.*Food.*'}}]}):
    cleaned_business = {}
    if row['categories'] is not None:
        # ignore business with no categories
        cleaned_business['business_id'] = row['business_id']
        cleaned_business['name'] = row['name']
        cleaned_business['city'] = row['city']
        cleaned_business['state'] = row['state']
        cleaned_business['review'] = row['stars'] # calculate weighted_review TODO
        cleaned_business['total_reviews'] = row['review_count']
        cleaned_business['categories'] = row['categories']
        business_records.append(cleaned_business)

In [79]:
business_df = pd.DataFrame(
    business_records,
    # In the loaded table, the column order reflects the order of the
    # columns in the DataFrame.
    columns=[
        "business_id",
        "name",
        "city",
        "state",
        "review",
        "categories"
    ])

business_df

,business_id,name,city,state,review,categories
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,Philadelphia,PA,4.0,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
1,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,Green Lane,PA,4.5,"Brewpubs, Breweries, Food"
2,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,Ashland City,TN,2.0,"Burgers, Fast Food, Sandwiches, Food, Ice Crea..."
3,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,Affton,MO,3.0,"Pubs, Restaurants, Italian, Bars, American (Tr..."
4,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,Nashville,TN,1.5,"Ice Cream & Frozen Yogurt, Fast Food, Burgers,..."
...,...,...,...,...,...,...
64611,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,Boise,ID,4.0,"Cafes, Juice Bars & Smoothies, Coffee & Tea, R..."
64612,1jx1sfgjgVg0nM6n3p0xWA,Savaya Coffee Market,Oro Valley,AZ,4.5,"Specialty Food, Food, Coffee & Tea, Coffee Roa..."
64613,WnT9NIzQgLlILjPT0kEcsQ,Adelita Taqueria & Restaurant,Philadelphia,PA,4.5,"Restaurants, Mexican"
64614,2O2K6SXPWv56amqxCECd4w,The Plum Pit,Aston,DE,4.5,"Restaurants, Comfort Food, Food, Food Trucks, ..."


#### b. group the restaurants by name

In [ ]:
# name_df = business_df.groupby('name').agg('number_outlets' : ('total_reviews', np.sum),'average_review' : ('review', np.mean))

In [70]:
# Output for names table
names_records = []
for j in business_collection.aggregate([{'$match' :  {'$or' : [{'categories' : {'$regex' : '.*Restaurants.*'}},{'categories' : {'$regex' : '.*Food.*'}}]}},
                                        {'$group' : {'_id' : '$name', 'average_review' : {'$avg' : '$stars'}, 
                                                     'total_reviews' : {'$sum' : '$review_count'},
                                                    'number of outlets' : {'$sum' : 1}}},
                                        {'$sort' : {'number of outlets' : -1}}]):
    names_records.append(j)

In [71]:
names_df = pd.DataFrame(
    names_records,
    # In the loaded table, the column order reflects the order of the
    # columns in the DataFrame.
    columns=[
        "_id",
        "average_review",
        "total_reviews",
        "number of outlets",
    ])
names_df

,_id,average_review,total_reviews,number of outlets
0,Starbucks,3.126381,20692,724
1,McDonald's,1.863442,17359,703
2,Dunkin',2.302941,9864,510
3,Subway,2.586057,4123,459
4,Taco Bell,2.154795,8325,365
...,...,...,...,...
43831,Ch. Cafeteria,5.000000,8,1
43832,Flava! Jamaican Soul Food Restaurant,3.000000,44,1
43833,DeLosa's Pizza,3.000000,268,1
43834,Three Palms Bar & Grill,4.500000,50,1


## 3. Load data into warehouse
https://cloud.google.com/bigquery/docs/samples/bigquery-load-table-dataframe 

In [6]:
from google.cloud import bigquery